In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

device = "cuda:0"

In [12]:
# specify how to quantize the model
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", quantization_config=quantization_config, device_map=device)
# model.to(device)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [14]:
import pandas as pd

df = pd.read_csv("../data/products/dataset_filtered.csv")
df = df.fillna("")
df.head()

,url,cluster1,cluster2,cluster3,main_image_url,other_images_urls,product_header,product_details,good_to_know_details,materials_details,image_url,label,image_path,x,y
0,https://www.ikea.com/us/en/p/strandmon-wing-ch...,Fabric armchairs,Armchairs,,https://www.ikea.com/us/en/images/products/str...,['https://www.ikea.com/us/en/images/products/s...,"Wing chair, Vibberbo black/beige",You can really loosen up and relax in comfort ...,,Materials & care\nMaterial\nFabric:\n100 % cot...,https://www.ikea.com/us/en/images/products/str...,0,../data/products/images/strandmon-wing-chair-v...,1.632644,-6.932771
1,https://www.ikea.com/us/en/p/havberg-armchair-...,Leather armchairs,Armchairs,,https://www.ikea.com/us/en/images/products/hav...,['https://www.ikea.com/us/en/images/products/h...,"Armchair and ottoman, Grann/Bomstad black",HAVBERG swivel armchair has an inviting and ti...,Grain leather is treated with a light protecti...,Materials & care\nMaterial\nOttoman\nSupportin...,https://www.ikea.com/us/en/images/products/hav...,1,../data/products/images/havberg-armchair-and-o...,7.518699,4.676394
2,https://www.ikea.com/us/en/p/soederhamn-cover-...,SÖDERHAMN covers,Sofa & armchairs covers,,https://www.ikea.com/us/en/images/products/soe...,[],"Cover for ottoman, Fridtuna light beige","This cover is made of Fridtuna, a sturdy cotto...",This product is an extra cover. Ottoman frame ...,Materials & care\nMaterial\nFabric:\n78 % cott...,https://www.ikea.com/us/en/images/products/soe...,2,../data/products/images/soederhamn-cover-for-o...,6.421540,-4.246849
3,https://www.ikea.com/us/en/p/finnala-cover-for...,FINNALA covers,Sofa & armchairs covers,,https://www.ikea.com/us/en/images/products/fin...,[],"Cover for armrest, Orrsta black-blue",Orrsta cover is woven from cotton and polyeste...,,Materials & care\nMaterial\n65% polyester (min...,https://www.ikea.com/us/en/images/products/fin...,3,../data/products/images/finnala-cover-for-armr...,-3.483738,0.698187
4,https://www.ikea.com/us/en/p/skansnaes-groenst...,Dining sets up to 4 seats,Dining sets,,https://www.ikea.com/us/en/images/products/ska...,['https://www.ikea.com/us/en/images/products/s...,"Table and 4 chairs, armrests brown/turquoise,",A statement piece for your dining area with fi...,Recommended for indoor use only.\nSeats 4-6.\n...,Materials & care\nMaterial\nExtendable table\n...,https://www.ikea.com/us/en/images/products/ska...,4,../data/products/images/skansnaes-groensta-tab...,16.184507,13.785761


In [15]:
chairslist = [
'Armchairs', 'Dining chairs', 'Outdoor chairs', 'Desk chairs', 'Armchairs & accent chairs', 
'Ottomans', 'Café furniture','Chairs', 
'Kids furniture',
]
print(len(df))
df = df[df.cluster2.isin(chairslist)].reset_index(drop=True)
print(len(df))

2691
821


In [16]:
texts = [
    f"Header: {row['product_header']}\nFurniture type: {row['cluster2']}\nDetails: {row['product_details'] + row['good_to_know_details'] + row['materials_details']}"
    for _, row in df.iterrows()
]

In [17]:
from tqdm.auto import tqdm

summ_results = []
for txt in tqdm(texts[400:]): 
    
    messages = [
        {"role": "user", "content": "You are product summarization assistant. You need to get the key bullet points about the product from the text. Some of the key information is style, upholstery, color, material, shape. If something is not nessesery just skip it. Ignore warranty and care instructions. You must write only bullet points. As short as possible."},
        {"role": "assistant", "content": "Ok! I will help you to summarize the product descriptions!"},
        {"role": "user", "content": f"The text: ```{txt}```"}
    ]
    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
    model_inputs = encodeds.to(device)

    generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    
    output = decoded[0].split("[/INST]")[-1].replace("</s>", "").strip()
    bullet_points = "\n".join([row for row in output.split("\n") if row.strip() and row.strip()[0] in ("*", "-", "+")])

    summ_results.append(bullet_points)

  0%|          | 0/421 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attentio

In [4]:
summ_results

NameError: name 'summ_results' is not defined